# Heidi Hendrickson's blsa protein prep_leap_ions workflow

### This demo outlines how to load in a PDB file of a protein and adjust it to be an AMBER compatible PDB file. Once it is an AMBER PDB file we run simulations with solvating the protein.

### For more information visit: https://pubs.acs.org/doi/abs/10.1021/acsbiomaterials.6b00447

<div class="alert alert-block alert-info">
    <b>Step 1:</b> <font color='black'>Load AMBER parameters and solvent boxes. </font></div>

<div class="alert alert-block alert-success">
    </b> <font color='black'>Load the ff14SB protein force field</font></div>

In [ ]:
source "leaprc.protein.ff14SB"

<div class="alert alert-block alert-success">
    </b> <font color='black'>Load water and ions for TIP3P box</font></div>

In [ ]:
source leaprc.water.tip3p

<div class="alert alert-block alert-success">
    </b> <font color='black'>Load in the general Amber Force field.</font></div>

In [ ]:
source leaprc.gaff

<div class="alert alert-block alert-info">
    <b>Step 2:</b> Scan the PDB file to generate a summary that will help us convert it to an AMBER PDB file.<font color='black'></font></div>

In [ ]:
scan = scanPdb data/bsla/4bhu.pdb

<div class="alert alert-block alert-info">
    <b>Step 3:</b> Scan and select a specific chain. We only want to use the C chain.<font color='black'></font></div>

In [ ]:
scanSelectChainIds scan { :C }

<div class="alert alert-block alert-info">
    <b>Step 4:</b> Run the following filters to create an AMBER compatible PDB file. Scan the file periodically to check if the filters have succeeded. <font color='black'></font></div>

In [ ]:
scanAmberCheck scan

<div class="alert alert-block alert-success">
    </b> <font color='black'>Ignore any HOH and Cl residues</font></div>

In [ ]:
scanIgnoreResidues scan { :HOH :CL }

<div class="alert alert-block alert-success">
    </b> <font color='black'>Rename residues MLY->LYS and MSE->MET</font></div>

In [ ]:
scanRenameResidues scan { { :MLY :LYS } { :MSE :MET } }

In [ ]:
scanAmberCheck scan

<div class="alert alert-block alert-success">
    </b> <font color='black'>Rename atoms in the MSE residues SE->SD</font></div>

In [ ]:
scanRenameAtoms scan { { :MSE { :SE :SD } } }

In [ ]:
scanAmberCheck scan

<div class="alert alert-block alert-success">
    </b> <font color='black'>Remove the given atoms from the MLY residue.</font></div>

In [ ]:
scanIgnoreAtoms scan { { :MLY { :HH22 :HH21 :HH23 :HH13 :HH12 :HH11 :CH2 :CH1} } }

<div class="alert alert-block alert-success">
    </b> <font color='black'>Run a final check</font></div>

In [ ]:
scanAmberCheck scan

<div class="alert alert-block alert-info">
    <b>Step 5:</b> We now load in the scan and the fixups we added as an Amber compatible PDB file <font color='black'></font></div>

In [ ]:
complex = loadPdb scan

<div class="alert alert-block alert-info">
    <b>Step 6:</b> We can now visualize the protein.<font color='black'></font></div>

In [ ]:
show complex

<div class="alert alert-block alert-info">
    <b>Step 7:</b> We need to create a box around the protein so that we can run simulations on it<font color='black'></font></div>

<div class="alert alert-block alert-success">
    </b> <font color='black'>Use AlignAxes to translate the geometric center of the protein to the origin and align
the principle axes of the ATOMs within the protein along the coordinate axes. This will minimize the box's volume.</font></div>

In [ ]:
alignAxes complex

In [ ]:
show complex :axes t

<div class="alert alert-block alert-success">
    </b> <font color='black'>The setBox command creates a periodic box around the protien UNIT, turning it into a periodic system for the simulation programs. It does not add any solvent to the system. The choice of "vdw" ensures that the box encloses the entirety of all atoms. </font></div>

In [ ]:
setBox complex "vdw"

<div class="alert alert-block alert-success">
    </b> <font color='black'>Move the geometric center of the matter to the origin</font></div>

In [ ]:
(chem:geometric-center complex)

<div class="alert alert-block alert-success">
    </b> <font color='black'>You can now visualize the adjusted complex</font></div>

In [ ]:
show complex

<div class="alert alert-block alert-info">
    <b>Step 8:</b>We want to run a simulation where we solvate the protein by adding water to the box.<font color='black'></font></div>

<div class="alert alert-block alert-success">
    </b> <font color='black'>The command 'resolvateBox' creates a checkpoint at this stage. Regardless of what else we add to the box after this step we can always return here and reset the box.</font></div>

<div class="alert alert-block alert-success">
    </b> <font color='black'>TIP3PBOX specifies that the solvent will be water and the numerical value represents the closeness of atoms in Angstroms</font></div>

In [ ]:
resolvateBox complex TIP3PBOX 8.0

<div class="alert alert-block alert-success">
    </b> <font color='black'>You can now visualize the solvated complex</font></div>

In [ ]:
show complex

<div class="alert alert-block alert-info">
    <b>Step 9:</b> After creating a copy of the complex we want to try adding different ions into the box<font color='black'></font></div>

In [ ]:
ion-complex = copy complex

In [ ]:
addIons ion-complex NA 0

In [ ]:
addIons ion-complex CL 0

<div class="alert alert-block alert-success">
    </b> <font color='black'>You can now visualize the ion-solvated complex</font></div>

In [ ]:
show ion-complex

<div class="alert alert-block alert-info">
    <b>Step 9:</b> Save the new AMBER compatible file with all of the simulation parameters<font color='black'></font></div>

<div class="alert alert-block alert-success">
    </b> <font color='black'>Use saveamberparm to save the AMBER topology and coordinate files for the UNIT into the
files named _topologyfilename_ and _coordinatefilename_ respectively. This will save all the parameters of this simulation so that it can be accessed again exactly as it is now.</font></div>

In [ ]:
saveamberparm ion-complex data/bsla/full_N_bsla_spytag_helix_solv12A.prmtop data/bsla/full_N_bsla_spytag_helix_solv12A.inpcrd

<div class="alert alert-block alert-success">
    </b> <font color='black'>Save this ion-complex as a new PDB file</font></div>

In [ ]:
savepdb ion-complex data/bsla/full_N_bsla_spytag_helix_solv12A.pdb

## All done!

edited by Ava Dremann 07-13-2022, 07-19-2022